<a href="https://colab.research.google.com/github/applejxd/colaboratory/blob/master/ml/TensorFlowCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")

TensorFlow version: 2.8.2


In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

print(x_train.shape)

(60000, 28, 28)


## 通常の分類問題として解く

In [3]:
# Dense レイヤ = アファインレイヤ (& 活性化関数)
simple_model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(50, activation='sigmoid'),
  tf.keras.layers.Dense(100, activation='sigmoid'),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax()
])
simple_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 50)                39250     
                                                                 
 dense_1 (Dense)             (None, 100)               5100      
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
 softmax (Softmax)           (None, 10)                0         
                                                                 
Total params: 45,360
Trainable params: 45,360
Non-trainable params: 0
_________________________________________________________________


In [4]:
print(x_train[:1].shape)

# 標本毎にロジットを計算
predictions = simple_model(x_train[:1]).numpy()
predictions

(1, 28, 28)


array([[0.091982  , 0.11054815, 0.07437743, 0.06012511, 0.12859702,
        0.09779611, 0.07461037, 0.09240244, 0.09742267, 0.17213868]],
      dtype=float32)

In [5]:
# 交差エントロピーによる損失関数
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

loss_fn(y_train[:1], predictions).numpy()

2.3052518

In [6]:
# 学習の構成を指定
simple_model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

In [7]:
# 学習を実施
simple_model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 1.7373 - accuracy: 0.7533
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 1.5430 - accuracy: 0.9280
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 1.5228 - accuracy: 0.9445
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 1.5127 - accuracy: 0.9526
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.5052 - accuracy: 0.9600


In [8]:
# 評価
simple_model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 1.5064 - accuracy: 0.9582 - 523ms/epoch - 2ms/step


[1.5064195394515991, 0.9581999778747559]

## CNN で解く

In [11]:
# kernel_size = filter_size
cnn_model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters=30, kernel_size=(5, 5), 
                         padding="valid", strides=1, activation="relu",
                         input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPool2D((2, 2)),
  tf.keras.layers.Flatten(input_shape=(12, 12)),
  tf.keras.layers.Dense(100, activation="relu"),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax()
])
cnn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 24, 24, 30)        780       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 30)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 4320)              0         
                                                                 
 dense_5 (Dense)             (None, 100)               432100    
                                                                 
 dense_6 (Dense)             (None, 10)                1010      
                                                                 
 softmax_2 (Softmax)         (None, 10)                0         
                                                      

In [12]:
# 学習の構成を指定
cnn_model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
# 学習を実施
cnn_model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 39s 20ms/step - loss: 1.5402 - accuracy: 0.9260
Epoch 2/5
1875/1875 [==============================] - 38s 20ms/step - loss: 1.4846 - accuracy: 0.9784
Epoch 3/5
1875/1875 [==============================] - 37s 20ms/step - loss: 1.4783 - accuracy: 0.9839
Epoch 4/5
1875/1875 [==============================] - 39s 21ms/step - loss: 1.4753 - accuracy: 0.9866
Epoch 5/5
1875/1875 [==============================] - 37s 20ms/step - loss: 1.4724 - accuracy: 0.9894


In [13]:
# 評価
cnn_model.evaluate(x_test,  y_test, verbose=2)

313/313 - 2s - loss: 1.4748 - accuracy: 0.9869 - 2s/epoch - 6ms/step


[1.4747909307479858, 0.9868999719619751]